# 🧑‍🍳 A kitchen that responds to your “I’m hungry” is more than feasible

# Plan our path

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.connectors.memory.chroma import ChromaMemoryStore

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI services used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
    kernel.add_text_embedding_generation_service("ada", AzureTextEmbedding("text-embedding-ada-002", api_key, endpoint))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-4", api_key, org_id))
#    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))
    kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

kernel.register_memory_store(memory_store=ChromaMemoryStore(persist_directory='mymemories'))
kernel.import_skill(sk.core_skills.TextMemorySkill())

/Users/johnmaeda/Documents/GitHub/pizzashop-and-ai/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1261ed5b0>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1261ed4f0>}

In [36]:
from semantic_kernel.planning import SequentialPlanner
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "./plugins-sk"
consultant_plugin = kernel.import_semantic_skill_from_directory(skills_directory, "FriendlyConsultant")

# create an instance of sequential planner.
planner = SequentialPlanner(kernel)

# the ask for which the sequential planner is going to find a relevant function.
# ask = """
# Create a business presentation about a hamburger restaurant."""
# ask = """
# Create a research interview about a hamburger restaurant business. The focus is on its strengths."""
# ask = """
# Create a business presentation about a hamburger restaurant business. The context is that has served the local community of Chicago for 43 years and is famous for chili burgers."""
ask = """
Create a marketing plan for a burger restaurant that has served the local community of Chicago for 43 years and is famous for chili burgers."""

# ask the sequential planner to identify a suitable function from the list of functions available.
plan = await planner.create_plan_async(goal=ask)

# ask the sequential planner to execute the identified function.
result = await plan.invoke_async()

for step in plan._steps:
    print("PLAN ",step.description, ":", step.skill_name + "." + step._function.name, step._parameters._variables, step._outputs)
#print("LENGTH: "+len(plan._steps))

for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    print("Input vars:", step._parameters._variables)
    print("Output vars:", step._outputs)
    if len(step._outputs) > 0:
        print( "  Output:", str.replace(result[step._outputs[0]],"\n", "\n        "))

    print()
# print("Final Answer:")
# print(result)

# #for step in plan._steps:
# #    print(step.description, ":", step.skill_name, step._function.name, step._state.__dict__)

# print("Expected Answer:")
# print(result)

# Example 3

In [22]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI services used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
#    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))
    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))
#    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-4", api_key, org_id))


In [24]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill
from semantic_kernel.planning import SequentialPlanner

kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

# create an instance of sequential planner.
planner = SequentialPlanner(kernel)

# the ask for which the sequential planner is going to find a relevant function.
ask = "What day of the week is today, all uppercase?"

# ask the sequential planner to identify a suitable function from the list of functions available.
plan = await planner.create_plan_async(goal=ask)

print(plan.description)

# ask the sequential planner to execute the identified function.
result = await plan.invoke_async()

for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    print("Input vars:", step._parameters._variables)
    print("Output vars:", step._outputs)
    if len(step._outputs) > 0:
        print( "  Output:", str.replace(result[step._outputs[0]],"\n", "\n        "))

    print()

for step in plan._steps:
    print(step.description, ":", step.skill_name, step._function.name, step._state.__dict__)

print("Expected Answer:")
print(result)


What day of the week is today, all uppercase?
Step: 0
Description: Get the current date.
Function: time.today
Input vars: {'input': ''}
Output vars: ['TODAY']
  Output: Monday, 14 August, 2023

Step: 1
Description: Get the current day of the week
Function: time.dayOfWeek
Input vars: {'input': '$TODAY'}
Output vars: ['DAY_OF_WEEK']
  Output: Monday

Step: 2
Description: Convert a string to uppercase.
Function: text.uppercase
Input vars: {'input': '$DAY_OF_WEEK'}
Output vars: ['RESULT__DAY_OF_WEEK_UPPERCASE']
  Output: MONDAY

Get the current date. : time today {'_variables': {'input': ''}, '_main_key': 'input'}
Get the current day of the week : time dayOfWeek {'_variables': {'input': ''}, '_main_key': 'input'}
Convert a string to uppercase. : text uppercase {'_variables': {'input': ''}, '_main_key': 'input'}
Expected Answer:
MONDAY


Another thing the pizza shop owner is figure out how to get more time back for themself. Often times that time gained can be used for the pizza shop to reinvest in the same way that money saved can be re-leveraged.

In [5]:
from semantic_kernel.planning import SequentialPlanner
from semantic_kernel.core_skills.text_skill import TextSkill

plugins_directory = "./plugins-sk"
writer_skill = kernel.import_semantic_skill_from_directory(plugins_directory, "LiterateFriend")
#text_skill = kernel.import_skill(TextSkill(), "TextSkill")

# create an instance of sequential planner.
planner = SequentialPlanner(kernel)

# the ask for which the sequential planner is going to find a relevant function.
ask = """
Tomorrow is Valentine's day. I need to come up with a poem. Translate the poem to French."""

# ask the sequential planner to identify a suitable function from the list of functions available.
plan = await planner.create_plan_async(goal=ask)

# ask the sequential planner to execute the identified function.
result = await plan.invoke_async()

for step in plan._steps:
    print(step.description, ":", step.skill_name + "." + step._function.name, step._parameters._variables, step._outputs)

for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    print("Input vars:", step._parameters._variables)
    print("Output vars:", step._outputs)
    if len(step._outputs) > 0:
        print( "  Output:", str.replace(result[step._outputs[0]],"\n", "\n        "))

    print()
print("Final Answer:")
print(result)

#for step in plan._steps:
#    print(step.description, ":", step.skill_name, step._function.name, step._state.__dict__)

print("Expected Answer:")
print(result)

Turn a scenario into a short and entertaining poem. : WriterSkills.ShortPoem {'input': '', 'topic': "Valentine's day"} ['POEM']
Translate the input into a language of your choice : WriterSkills.Translate {'input': '$POEM', 'language': 'French'} ['RESULT__FINAL_POEM']
Step: 0
Description: Turn a scenario into a short and entertaining poem.
Function: WriterSkills.ShortPoem
Input vars: {'input': '', 'topic': "Valentine's day"}
Output vars: ['POEM']
  Output: On Valentine's Day, full of cheer,
        Cupid's arrows fly far and near.
        But my date was a cat,
        In a little red hat,
        Guess it's love, but just not the right gear!

Step: 1
Description: Translate the input into a language of your choice
Function: WriterSkills.Translate
Input vars: {'input': '$POEM', 'language': 'French'}
Output vars: ['RESULT__FINAL_POEM']
  Output: Le jour de la Saint-Valentin, plein de joie,
        Les flèches de Cupidon volent de près et de loin.
        Mais mon rendez-vous était un chat